In [ ]:
import glob
import cv2 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import imutils
import copy
from scipy.ndimage.filters import  gaussian_filter
import math
import skimage.transform as sktr


### Loading images

In [ ]:
paths = [file for file in glob.glob('./inputs/P4/d/*.png')]
images = []
for path in paths:
    img = cv2.imread(path)
    images.append(img)
    

### Fourier

In [ ]:
def get_fourier(img):
    img = img.copy()
    img_dft = cv2.dft(np.float32(img),flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(img_dft)
    magnitude, angle = cv2.cartToPolar(dft_shift[:, :, 0],dft_shift[:, :, 1])
    magnitude = np.log(magnitude)
    
    return magnitude, angle


def pattern_remover(img,p=0.65,center=25):
    img = img.copy()
    out1 = img.copy()
    dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    magnitude, angle = cv2.cartToPolar(dft_shift[:, :, 0],dft_shift[:, :, 1])
    magnitude = np.log(magnitude)
    out2 = magnitude.copy()

    _max = max(magnitude.flatten())

    for row in range(magnitude.shape[0]):
        for col in range(magnitude.shape[1]):
            if row > magnitude.shape[0]//2 - center and row < magnitude.shape[0]//2 + center and col > magnitude.shape[1]//2 - center and col < magnitude.shape[1]//2 + center :
                continue 

            if magnitude[row,col] >= _max*p :
                magnitude[row,col] = 0 

    out3 = magnitude.copy()
    i_magnitude = np.exp(magnitude)
    i_dft_shift = cv2.polarToCart(i_magnitude,angle)
    i_dft_shift = np.stack([i_dft_shift[0],i_dft_shift[1]],axis=2)
    i_dft_shift = np.fft.ifftshift(i_dft_shift)
    i_dft = cv2.idft(i_dft_shift, flags=cv2.DFT_SCALE)
    out4 = cv2.magnitude(i_dft[:,:,0],i_dft[:,:,1])
    
    return out1,out2,out3,out4
    


### cubes proc

In [ ]:
Codes = [cv2.COLOR_BGR2YCrCb,None,None,None]
I_Codes = [cv2.COLOR_YCrCb2RGB,None,None,None]
Param = [ [[0.52,17],[0.55,17],[0.57,17]],None,None,None]
tags = ["YCrCb" ,None,None,None]
plt.rcParams['figure.figsize'] = [16, 10]

tmps = []
for Index in range(0,2) :
    r,c = images[0].shape[:2]
    if Index == 0 :
        img_bgr = images[0][:,:c//2,:].copy()
    else:
        img_bgr = images[0][:,c//2:,:].copy()
    Index = 0 
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")
    tmps.append(img_s)

    plt.show()


plt.rcParams['figure.figsize'] = [16,8]
plt.subplot(1,2,1)
plt.imshow(images[0][:,:,::-1])
plt.title("Input Image")
plt.axis("off")


plt.subplot(1,2,2)
plt.imshow(cv2.hconcat(tmps))
plt.title("Merged Filtered parts")
plt.axis("off")

plt.show()


In [ ]:
Codes = [cv2.COLOR_BGR2YCrCb,None,None,None]
I_Codes = [cv2.COLOR_YCrCb2RGB,None,None,None]
Param = [ [[0.55,17],[0.57,17],[0.59,17]],None,None,None]
tags = ["YCrCb" ,None,None,None]
plt.rcParams['figure.figsize'] = [16, 10]


for Index in range(0,1) :
    img_bgr = images[Index].copy()
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")
    plt.show()




### hearts_2 proc

In [ ]:
Codes = [None,cv2.COLOR_BGR2YCrCb,None,None]
I_Codes = [None,cv2.COLOR_YCrCb2RGB,None,None]
Param = [None, [[0.59,20],[0.57,20],[0.6,20]],None,None]
tags = [None,"YCrCb" ,None,None]
plt.rcParams['figure.figsize'] = [16, 10]

tmps = []
for Index in range(0,2) :
    r,c = images[1].shape[:2]
    if Index == 0 :
        img_bgr = images[1][:r//2,:c//2,:].copy()
    else:
        img_bgr = images[1][:r//2,c//2:,:].copy()
        
    Index = 1 
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")
    tmps.append(img_s)

    plt.show()


plt.rcParams['figure.figsize'] = [16,8]
plt.subplot(1,2,1)
plt.imshow(images[1][:,:,::-1])
plt.title("Input Image")
plt.axis("off")

m1 = cv2.hconcat(tmps)
m2 = cv2.hconcat(tmps[::-1])
res = cv2.vconcat([m1,m2])
plt.subplot(1,2,2)
plt.imshow(res)
plt.title("Merged Filtered parts")
plt.axis("off")

plt.show()




In [ ]:
Codes = [None,cv2.COLOR_BGR2YCrCb,None,None]
I_Codes = [None,cv2.COLOR_YCrCb2RGB,None,None]
Param = [None, [[0.59,20],[0.57,20],[0.6,20]],None,None]
tags = [None,"YCrCb" ,None,None]
plt.rcParams['figure.figsize'] = [16, 10]

tmps = []
for Index in range(1,2) :

    img_bgr = images[Index].copy() 
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")
    plt.show()



### Spheres_2 proc

In [ ]:
Codes = [None,None,cv2.COLOR_BGR2YCrCb,None]
I_Codes = [None,None,cv2.COLOR_YCrCb2RGB,None]
Param = [ None,None,[[0.55,18],[0.6,18],[0.61,18]],None]
tags = [None,None,"YCrCb" ,None]
plt.rcParams['figure.figsize'] = [16, 10]

for Index in range(2,3) :
    img_bgr = images[Index].copy()
    r,c = img_bgr.shape[:2]
    img_bgr = img_bgr[:r//2,:c//2,:]
    
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")


    plt.show()
    break
    
plt.rcParams['figure.figsize'] = [16,8]
plt.subplot(1,2,1)
plt.imshow(images[2][:,:,::-1])
plt.title("Input Image")
plt.axis("off")


plt.subplot(1,2,2)
m1 = cv2.hconcat([img_s,img_s[:,::-1,:]])
res = cv2.vconcat([m1,m1[::-1,:,:]])
plt.imshow(res)
plt.title("Merged Filtered parts")
plt.axis("off")

plt.show()


In [ ]:
Codes = [None,None,cv2.COLOR_BGR2YCrCb,None]
I_Codes = [None,None,cv2.COLOR_YCrCb2RGB,None]
Param = [ None,None,[[0.55,18],[0.6,18],[0.61,18]],None]
tags = [None,None,"YCrCb" ,None]
plt.rcParams['figure.figsize'] = [16, 10]

for Index in range(2,3) :
    img_bgr = images[Index].copy()
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")


    plt.show()
    break
    
